In [1]:
TE = joinpath(tempdir(), "OnePiece.test", "")

if isdir(TE)

    rm(TE; recursive = true)

end

mkdir(TE)

"/var/folders/mg/30npdf0s1z36txb2qvgylrzh0000gn/T/OnePiece.test/"

In [2]:
using Revise
using BenchmarkTools

---

In [3]:
pam = dirname(@__DIR__)

pap = joinpath(pam, "Project.toml")

pas = joinpath(pam, "src")

pat = joinpath(pam, "test")

;

In [4]:
using TOML

In [5]:
to = TOML.parse(read(pap, String))

mo = "OnePiece"

@assert to["name"] == splitext(basename(pam))[1] == mo

## Make src/OnePiece.jl 

In [6]:
su_ = [
    "io/table",
    "io/fcs",
    "io/gct",
    "io/gmt",
    #"io/pandas",
    "extension/constant",
    "extension/dict",
    "extension/vector",
    "extension/string",
    "extension/path",
    "templating",
    "informatics/tensor",
    "informatics/geometry",
    "informatics/normalization",
    "informatics/statistics",
    "informatics/significance",
    "informatics/information",
    "emoji",
    "figure",
    "tensor_function",
    "feature_by_sample",
    "gene",
    "feature_set_enrichment",
]

;

In [7]:
dis_ = Set(vcat([splitpath(su) for su in su_]...))

di_ = Set(vcat([wa[2] for wa in walkdir(pas)]...))

symdiff(dis_, di_)

Set{String} with 1 element:
  "pandas"

In [8]:
using JSON
using OrderedCollections

In [9]:
tr = OrderedDict()

tr[mo] = OrderedDict()

cu = tr[mo]

for su in su_

    di_ = splitpath(su)

    n_di = length(di_)

    for (id, di) in enumerate(di_)

        if id < n_di

            if !haskey(cu, di)

                cu[di] = OrderedDict()

            end

            cu = cu[di]

        else

            jl_ = [fi for fi in readdir(joinpath(pas, su)) if occursin(r"\.jl$", fi)]

            cu[di] = [joinpath(su, jl) for jl in jl_]

            cu = tr[mo]

        end

    end

end

JSON.print(tr, 2)

{
  "OnePiece": {
    "io": {
      "table": [
        "io/table/_.jl",
        "io/table/read.jl",
        "io/table/write.jl"
      ],
      "fcs": [
        "io/fcs/_.jl",
        "io/fcs/read.jl"
      ],
      "gct": [
        "io/gct/_.jl",
        "io/gct/read.jl"
      ],
      "gmt": [
        "io/gmt/read.jl"
      ]
    },
    "extension": {
      "constant": [
        "extension/constant/ALPHABET.jl",
        "extension/constant/CARD.jl"
      ],
      "dict": [
        "extension/dict/_.jl",
        "extension/dict/merge.jl",
        "extension/dict/read.jl",
        "extension/dict/sort_recursively!.jl",
        "extension/dict/summarize.jl",
        "extension/dict/symbolize_key.jl",
        "extension/dict/view.jl",
        "extension/dict/write.jl"
      ],
      "vector": [
        "extension/vector/get_longest_common_prefix.jl",
        "extension/vector/get_order.jl",
        "extension/vector/is_in.jl",
        "extension/vector/sort_like.jl"
      ],
      "string

In [10]:
function write_line(io, st)

    return write(io, string(st, "\n"))

end

write_line (generic function with 1 method)

In [11]:
function write_branch(io, id, va)

    sp = "  "^id

    if va isa OrderedDict

        for (ke, va2) in va

            write_line(io, string(sp, "module ", ke))

            write_branch(io, id + 1, va2)

            write_line(io, string(sp, "end"))

        end

    else

        for li in va

            write_line(io, string(sp, "include(\"", li, "\")"))

        end

    end


end

write_branch (generic function with 1 method)

In [12]:
open(joinpath(pas, string(mo, ".jl")), "w") do io

    write_branch(io, 0, tr)

end

In [13]:
using OnePiece

WebIO._IJuliaInit()

┌ Info: Precompiling OnePiece [1fe83854-81c0-42f7-afc9-71ba9af673ca]
└ @ Base loading.jl:1423


## Test submodules

In [ ]:
for su in su_

    nb = joinpath(pat, string(su, ".ipynb"))

    println("Running ", nb)

    run(
        `jupyter-nbconvert --execute --ExecutePreprocessor.timeout=-1 --clear-output --inplace --log-level 0 $nb`,
    )

end

---

In [ ]:
rm(TE; recursive = true)